## *DISCLAIMER*
<p style="font-size:16px; color:#117d30;">
 By accessing this code, you acknowledge the code is made available for presentation and demonstration purposes only and that the code: (1) is not subject to SOC 1 and SOC 2 compliance audits; (2) is not designed or intended to be a substitute for the professional advice, diagnosis, treatment, or judgment of a certified financial services professional; (3) is not designed, intended or made available as a medical device; and (4) is not designed or intended to be a substitute for professional medical advice, diagnosis, treatment or judgement. Do not use this code to replace, substitute, or provide professional financial advice or judgment, or to replace, substitute or provide medical advice, diagnosis, treatment or judgement. You are solely responsible for ensuring the regulatory, legal, and/or contractual compliance of any use of the code, including obtaining any authorizations or consents, and any solution you choose to build that incorporates this code in whole or in part.
</p>



# Production Performance - Batch Processing 
<h2><span style="color: #117d30;"> Using Synapse SQL Table</span></h2>

# Overview
Consider a typical industry scenario in which a conveyor belt scans or checks the quality of manufactured products and categorizes them into three categories viz. $Good$ OR $Snag$ OR $Reject$. The whole process gets saved to a local database and gets pushed to the Synapse SQL Pool instance every night.

In this notebook, we will batch process the data and we will visualize data to compare performances of different machines. See the workflow below.

# Workflow

![Without HTAP workflow](https://dreamdemostorageforgen2.blob.core.windows.net/mfgdemocontainer/notebook-images/htap-image)

### Scenario
As explained above, the data gets dumped into Synapse SQL Server instance every night, after which we can analyze the data to derive insights.

Table Details :

**TABLE: ** = $dbo.mfgMesQuality$

**JDBCDATABASE:** = $ManufacturingDW$

**Dataset**: We have following quality data for 8 machines in our Synapse SQL table of 6 days **(24th June 2019 to 29th June 2019)** :
- Overall Good/Snag/Reject.
- Overall Average of Good/(Good+Snag+Reject)
- Timestamp

***Database contains data from 24th June 2019 to 29th June 2019.***

### Tools/Techniques: 
- Connecting to Synapse SQL database using spark.
- Visualization Using Matplotlib


### Notebook Organization 

This notebook uses the Spark to pull data from Synapse SQL Server Instance and visualize it. 


In [3]:
%%spark
// Let's load latest batch of Telemetry data from Azure Synapse table. To do the same, we use 
// Scala syntax to read data from an Azure Synapse table and create a temporary view of this table.
// We also specify Azure Cosmos DB Container, which is mfg-mes-quality in this case.

var df_ProductionPerformanceBatch = spark.read.sqlanalytics("#SQL_POOL_NAME#.dbo.mfgMonthlyProduction")
df_ProductionPerformanceBatch.createOrReplaceTempView("ProductionPerformanceBatch")

df_ProductionPerformanceBatch: org.apache.spark.sql.DataFrame = [ProductionMonth: timestamp, MachineInstance: string ... 5 more fields]

In [4]:
# To proceed further for performing analysis in Notebook, we need to import few of Python's data processing 
# libraries. Additionally, we will use matplotlib library to plot the required charts.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
from datetime import datetime

In [5]:
# We use PySpark to load data from the temporary view created above into a Spark DataFrame.
# Then we convert the DataFrame into a Pandas DataFrame, which 
# in combination with Matplotlib provides an extensive set of tools for creating visualizations.

# reading from a Temp View
df_ProductionPerformanceBatch = sqlContext.table("ProductionPerformanceBatch")

# getting Pandas DataFrame for data exploration and visualization      
df_ProdPerfBatch = df_ProductionPerformanceBatch.toPandas()

# displaying product quality data from first 10 machines
display(df_ProdPerfBatch.head())

Note : Choosing data from a specific date range


In [6]:
# To better understand the data, we perform grouping on basis of:
# 1. Date
# 2. Machine Instance

# Post grouping, let's see how data looks by displaying entire Aggregated DataFrame.

# converting timestamp to datetime format
df_ProdPerfBatch['ProductionMonth'] = pd.to_datetime(df_ProdPerfBatch['ProductionMonth'])

# changing Pandas DataFrame index to ProductionMonth
df_ProdPerfBatch.set_index(df_ProdPerfBatch["ProductionMonth"],inplace=True)

# grouping data by machine and aggregating by daily mean
df_AvgProdPerfBatch = df_ProdPerfBatch.groupby(["MachineInstance"], sort=False).resample('D').mean().reset_index()
display(df_AvgProdPerfBatch.head())

## Tabulating the daywise averages for machines
- The above dataFrame (df_AvgProdPerfBatch) has aggregated data per day for each machine.

-  We will showcase this using a Table.

- Lets keep a **threshold of 95%**. whenever a machine's daily average falls below 95% it will be marked as critical.

In [7]:
# It's time to visualize our DataFrame and identify what fits our requirements from a business perspective.
# As a business rule, threshold of 95% is minimum acceptable Quality.
# Lower than 95% threshold indicates a problem which may need further inspection.
# For visualizing data, we use matplotlib to draw a table 
# with yellow background indicating a problem.
# plotting a table to see percentage of daily quality data for each machine


rgb_normal = '#01B8AA'
rgb_alert = '#E2B803'

# plotting the table
cols = df_AvgProdPerfBatch['ProductionMonth'].unique()
colNames = [dt.strftime("%d %b") for dt in pd.to_datetime(cols)]
index = df_AvgProdPerfBatch['MachineInstance'].unique()
index.sort()
df_AvgMachineProduction = pd.DataFrame(index=index,columns=cols)
df_TableColors = pd.DataFrame(index=index,columns=cols)

# initialising data and colours for table
for i,row in df_AvgProdPerfBatch.iterrows():
    r = row['MachineInstance']
    c = row['ProductionMonth']
    avg_str = str(round(row['Avg'],2))[:5]
    if len(avg_str) < 5:
        avg_str+='0'
    df_AvgMachineProduction.loc[r,c] = avg_str
    if(row['Avg']<95):
        df_TableColors.loc[r,c]=rgb_alert
    else:
        df_TableColors.loc[r,c]=rgb_normal

# plotting Table
fig, ax = plt.subplots(figsize = (4,4))
ax.axis('off')
ax.axis('tight')
the_table = ax.table(cellText=np.array(df_AvgMachineProduction),cellColours=np.array(df_TableColors), cellLoc ='center', colWidths=[0.15]*len(cols) ,rowLabels=index,colLabels=colNames,loc='center')
the_table.auto_set_font_size(False)
the_table.set_fontsize(12)
the_table.scale(3,2)

fig.tight_layout()
plt.show()

## Stacked Bar Chart Visualization
- Quality data contains number of items processed into three categories :
    - Good
    - Snag
    - Reject
- We will plot it as a Stacked bar chart.

In [8]:
# Once we have plotted Quality averages, we need to see distribution between elements of Quality, viz, Good, Snag and Reject.
# To plot elements of Quality, we use Horizontally Stacked Bar Charts, with customized starting point to see all values.


# Plotting a horizontal stacked bar chart to show daily 'Good', 'Snag', and 'Reject' outputs for each machine
# Note: These numbers represent the production calculated by averaging each batch in a day and summed across all machines

df_stackPlot = df_ProdPerfBatch.resample('D').sum().reset_index()

y = df_stackPlot['ProductionMonth']
df_Good = df_stackPlot['Good']
df_Snag = df_stackPlot['Snag']
df_Reject = df_stackPlot['Reject']

#Stack Bar Plot 
fig, axs = plt.subplots(figsize=(13,7))
y_index = np.arange(len(y))

color_good = ["#02B480"]
color_snag= ["#E2B803"]
color_reject = ["#FD6C53"]

colNames = [dt.strftime("%d-%b") for dt in pd.to_datetime(y)]

bar1 = axs.barh(y_index, df_Good, alpha=0.7, color = color_good, linewidth = 2, edgecolor = color_good, tick_label=colNames)
bar2 = axs.barh(y_index, df_Snag,left=df_Good, alpha=0.7,  color = color_snag,linewidth= 2,edgecolor=color_snag,tick_label=colNames)
bar3 = axs.barh(y_index, df_Reject, left =(df_Snag+df_Good), alpha=0.7,  color = color_reject,linewidth= 2,edgecolor=color_reject,tick_label=colNames)

# values for setting x limit
min_val = min(df_Good)
max_val = max(df_Good+df_Snag+df_Reject)
diff = max_val - min_val
x_min = min_val-diff/2
x_max = max_val+diff/2
axs.set_xlim(x_min, x_max)

# writing text in bar graphs
for i, patch in enumerate(bar1.get_children()):
    bl = patch.get_xy()
    x_cord = x_min/2 +  0.5*patch.get_width() + bl[0]
    y_cord = 0.5*patch.get_height() + bl[1]
    axs.text(x_cord,y_cord, str(int(round(df_Good[i]))), ha='center',va='center')

for i, patch in enumerate(bar2.get_children()):
    bl = patch.get_xy()
    x_cord = 0.5*patch.get_width() + bl[0]
    y_cord = 0.5*patch.get_height() + bl[1]
    axs.text(x_cord,y_cord, str(int(round(df_Snag[i]))), ha='center',va='center')

for i, patch in enumerate(bar3.get_children()):
    bl = patch.get_xy()
    x_cord = 0.5*patch.get_width() + bl[0]
    y_cord = 0.5*patch.get_height() + bl[1]
    axs.text(x_cord,y_cord, str(int(round(df_Reject[i]))), ha='center',va='center')

axs.set_xlabel('Production')
axs.set_ylabel('Date')
fig.suptitle('Overall Production')
plt.legend((bar1[0], bar2[0], bar3[0]), ('Good', 'Snag', 'Reject'))
plt.show()

## Bar Chart Visualization
- To get an insight of which machine is behaving abnormaly or showing poor performance, let's compare all of them using a bar chart.
- We are calculating mean of each machine's production AVG for all seven days.

In [9]:
# Up until now, we have been visualizing daily metrics. To interprete exact problem,
# we also need to see how each machine Quality output looks like. By combining data from both
# sides, we can pinpoint a specific machine that appears to be under performing.


# plotting bar graph chart to show total quality of production for each machine


#Bar Chart
df_MachineTotalAvg = df_ProdPerfBatch.groupby(["MachineInstance"], sort=True).mean()

# creating a list of avg production of each machine
performance = df_MachineTotalAvg['Avg']
names= df_MachineTotalAvg.index.tolist()

# plotting bar graph
plt.figure(figsize=(13,7))
y_pos = np.arange(len(names))

edge_colour = ["#FFA132"]
color_map = ['#FFA132', '#FFAA46', '#FFB35A', '#FFBC6D', '#FFC581', '#FFCE94', '#FFD7A8']
plt.barh(y_pos, performance, align='center', alpha=1,  color = color_map, edgecolor=color_map)
plt.yticks(y_pos, names)

perf_min = min(performance)
perf_max = max(performance)
perf_diff = (perf_max - perf_min)/2
plt.xlim(perf_min-perf_diff, perf_max+perf_diff)

plt.xlabel('Percentage')
plt.ylabel('Machines')
plt.title('Production Performance of Machines')

plt.show()

In [10]:
# Let's visualize correlation between Quality of production of all machines per day against number of Good pieces produced per day.
# For doing the same, we are going to use Date on X-Axis. Stackbars will represent Average quality output of all machines.
# Line overlapped over Stackbars will represent number of Good pieces produced across all machines for that day.


# plotting graph to show the correlation between daily overall quality and average quality of all batches in a day 

df_barPlot = df_AvgProdPerfBatch.set_index(df_AvgProdPerfBatch["ProductionMonth"]).resample('D').mean().reset_index()
df_linePlot = df_AvgProdPerfBatch.set_index(df_AvgProdPerfBatch["ProductionMonth"]).resample('D').sum().reset_index()

# left side y axis
y = df_barPlot['Avg'].tolist()
# right side y axis
y2 = df_linePlot['Good'].tolist()
# common x axis
x = df_barPlot['ProductionMonth'].tolist()
x_Names = [dt.strftime("%d-%b") for dt in pd.to_datetime(x)]
x_pos = np.arange(len(x))

# plotting bar graph
fig, ax1 = plt.subplots(figsize=(13,7))
# color_map = ["#E2B803"]
color_map = ['#FFA132', '#FFAA46', '#FFB35A', '#FFBC6D', '#FFC581', '#FFCE94', '#FFD7A8']

# bar plot
p1 = ax1.bar(x_pos, y, align='center', alpha=1, color = color_map,tick_label=x_Names)
ax1.set_xticks(x_pos,x_Names)

# setting y limits
y_min = min(y)
y_max = max(y)
y_diff = (y_max - y_min)
ax1.set_ylim(y_min-y_diff, y_max+y_diff/2)

# line plot
line_color = '#02C480'
ax2 = ax1.twinx()
p2 = ax2.plot(x_pos,y2,color = line_color)
ax2.set_ylabel('Quality')

plt.legend((p1[0], p2[0]), ('Quality', 'Good'))
ax1.set_ylabel('Average Percentage of Availability')
ax1.set_xlabel('Date')
fig.suptitle("Overall Quality")

plt.show()

## Visualizing Average Availability for all Machines till 29th June Midnight 


In [11]:
# Let us analyse the latest available data. Since we are conducting ETL operations in daily batches,
# the latest available data is only upto the time of last batch.
# Let's plot a graph showing average Quality output of all machines for the last few hours.

df_AvailabilityChartData = df_ProductionPerformanceBatch.toPandas()
df_AvailabilityChartData.sort_values('ProductionMonth', inplace=True)

# last 40 records
df_RecentRecords = df_AvailabilityChartData.tail(60)
df_RecentRecords = df_RecentRecords.groupby(["ProductionMonth"], sort=True).mean()

performance = df_RecentRecords['Avg'].tolist()
names = df_RecentRecords.index.tolist()
perf_min = min(performance)
perf_max = max(performance)
perf_diff = (perf_max - perf_min)/2

# plotting bar graph
plt.figure(figsize=(13,7))
y_pos = np.arange(len(names))
colNames = [dt.strftime("%H:%M") for dt in pd.to_datetime(names)]
# color_map = ['#FFA150','#FFA132', '#FFAA46', '#FFB35A', '#FFBC6D', '#FFC581', '#FFCE94', '#FFD7A8']
color_map=['#FB7351','#02B480','#02B480','#02B480','#F98444','#02B480','#02B480','#02B480','#F98444','#02B480','#02B480','#02B480']
plt.bar(y_pos, performance, align='center', alpha=1, color = color_map, edgecolor=color_map)
plt.xticks(y_pos, colNames)
plt.ylim(perf_min-perf_diff, perf_max+perf_diff)
plt.ylabel('Average Percentage of Quality')
plt.xlabel('Time of the day ({})'.format(pd.to_datetime(names)[0].strftime("%d-%b")))
plt.title("Machines Average Quality (Till 29th June Midnight)")

plt.show()